## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- сравнить с точностью полносвязной сети для этой выборки
    </li>

Вариант 2. (сложный)
- реализовать нейронную сеть или стек из сверток (Concatenate) на сifar10.
- оценить рост точности при увеличении ширины сети (больше ядер), больше нитей.
- оценить рост точности при увеличении глубины сети (больше слоев)
    </li>
</ol>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical


In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
normalizer = Normalization()
normalizer.adapt(x_train)
x_train = normalizer(x_train)
x_test = normalizer(x_test)
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

In [4]:
x_train = tf.reshape(x_train,(x_train.shape[0], 28,28,1))
x_test = tf.reshape(x_test,(x_test.shape[0], 28,28,1))



In [5]:
x_train.shape

TensorShape([60000, 28, 28, 1])

In [6]:
def  fit_nn(x_train, y_train, hiden_shapes=[[128,64,32]], cv=3):
    results = []
    for shape in hiden_shapes:
        cv_results =[]
        for i in range(cv):
            model = Sequential()

            model.add(layers.Conv2D(64, kernel_size=(7, 7), strides=4, activation='relu', input_shape=(28,28,1), padding="same"))
            model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2, padding='same'))

            model.add(layers.Conv2D(64, kernel_size=(5, 5), strides=4, activation='relu', padding='same'))
            model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2, padding='same'))


            model.add(layers.Conv2D(64, kernel_size=(3, 3), strides=4, activation='relu', padding='same'))
            model.add(layers.Conv2D(64, kernel_size=(3, 3), strides=4, activation='relu', padding='same'))
            model.add(layers.Conv2D(32, kernel_size=(3, 3), strides=4, activation='relu', padding='same'))

            model.add(layers.Flatten())

            for i in range(len(shape)):
                model.add(layers.Dense(shape[i], activation='relu'))
        
            model.add(layers.Dense(10, activation='softmax'))

            model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
            hist = model.fit(x=x_train,y=y_train, epochs=40, batch_size=1024, validation_data=(x_test, y_test), verbose=0)

        cv_results.append(round(hist.history['val_accuracy'][-1], 4) * 100)
        results.append((shape, np.mean(cv_results)))
    return results

In [7]:
max_shape = [i for i in range(50, 400, 100)]
hiden_shapes1 = [[i, i//2] for i in max_shape]

In [8]:
%%time
 result1 = fit_nn(x_train, y_train, hiden_shapes=hiden_shapes1, cv=3)

Wall time: 18min 30s


In [9]:
hiden_shapes2 = [[280, 150], [270,  150, 80], [260, 150, 80, 50, 25], [230, 130, 110, 100 , 90, 80, 60, 40, 30, 20 ]]

In [10]:
%%time
result2 = fit_nn(x_train, y_train, hiden_shapes=hiden_shapes2, cv=3)

Wall time: 19min 23s


In [12]:
result1

[([50, 25], 88.28),
 ([150, 75], 88.57000000000001),
 ([250, 125], 88.58),
 ([350, 175], 88.85)]

Как мы видим очень простая сеть справляется практически так же хорошо как и очень сложная.

In [13]:
result2

[([280, 150], 88.42),
 ([270, 150, 80], 89.11),
 ([260, 150, 80, 50, 25], 88.57000000000001),
 ([230, 130, 110, 100, 90, 80, 60, 40, 30, 20], 87.99)]

Тут тоже сети справляются примерно одинаково, но количество уровней при той же сложности ухудшает результат.
Во втором домашнем задании лучший результат был accuracy 87,7%(правда это не кроссвалидированный результат, так что может в среднем она выдает меньшую точность)  хуже  этой сверточной сети, но не сильно.
Честно говоря я не совсем понимаю как настраивать такие сети, ведь в них очень много параметров, и сети работают нестабильно. что бы сделать нормальный грид серч с кросс валидацией нужно тратить чудовещное количество времени.